In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# WellnessMate: Multi-Agent Health & Wellness Assistant

This project is my capstone submission for the Kaggle x Google AI Agents Intensive.

## Key AI Agents Concepts Demonstrated

In this capstone project, I apply multiple core concepts from the **Kaggle x Google AI Agents Intensive**. The project demonstrates at least **five** of the required agentic capabilities:

---

### **1. Multi-Agent System (Sequential + Loop Behaviour)**  
This project uses a structured multi-agent architecture:

- **PlannerAgent** – understands the user’s query and creates an analysis plan.
- **AnalyticsAgent** – runs the actual data analysis using tools (statistics, trends, correlations).
- **CoachAgent** – turns numerical results into personalised wellness recommendations.

The agents run in a **sequential pipeline**:

> User → PlannerAgent → AnalyticsAgent → CoachAgent → Final Output

I also show a simple **loop-style behaviour**, where the system can run multiple iterations to update or refine recommendations with new data.

---

### **2. Tools (Custom Tools + Code Execution)**  
The agents do not directly manipulate the raw data.  
Instead, they call a registry of **custom tools**, such as:

- `filter_by_date()`
- `compute_basic_stats()`
- `compute_correlations()`
- `recent_trend()`
- (optional) `generate_plot()` using Python execution

This follows the tool-calling pattern taught in the course, where agents “take action” by calling tools rather than embedding all logic internally.

---

### **3. Sessions & Memory (Short-Term + Long-Term)**  
This system uses **two layers of memory**:

- **Session memory**  
  Stores the current conversation state and intermediate results.

- **Long-term memory**  
  Implemented as a simple JSON store that saves recurring wellness insights, such as patterns between sleep and mood.

This demonstrates how agents can use both **short-term session context** and **persistent long-term knowledge**.

---

### **4. Observability (Logging, Tracing, Metrics)**  
The system logs every major action into a shared **trace**, including:

- which agent ran,
- which tools were used,
- timestamps for each step.

I also track simple **metrics**:

- number of tool calls,
- runtime,
- number of generated recommendations.

This reflects the observability practices taught in the course (logging, tracing, metrics).

---

### **5. Agent Evaluation**  
I include lightweight evaluation checks:

- verifying dataset loading,
- checking that statistical tools produce expected outputs,
- ensuring that at least one recommendation is generated for normal queries.

This shows a basic but clear approach to **agent quality evaluation**, as required in the capstone.


## System Architecture Overview

### **1. High-Level Concept**
The idea behind WellnessMate is simple:

> “Given a user’s daily wellness logs (sleep, steps, mood, hydration, workouts),  
> how can an AI agent system understand the user’s questions, analyze their wellness patterns,  
> and suggest small, personalised improvements?”

Rather than using a single large script, the project uses multiple collaborating **agents**, each responsible for part of the workflow.

---

### **2. Agents in the System**

#### **1. PlannerAgent**
- Reads the user’s natural language question.  
- Determines what type of analysis is needed (trends, correlations, general overview, etc.).  
- Produces a simple **plan** describing which steps and tools should run.

#### **2. AnalyticsAgent**
- Takes the plan from the PlannerAgent.  
- Uses tools to analyze the wellness dataset:
  - compute basic statistics,
  - extract recent trends,
  - compute correlations between variables.

#### **3. CoachAgent**
- Takes the analysis results.  
- Converts numerical data into practical, personalised wellness recommendations.  
- Stores useful insights into **long-term memory** (Memory Bank style).

Together, these agents work in a **sequential pipeline**:
> Planner → Analytics → Coach

---

### **3. Tools Layer**
To follow the agentic pattern from the course, the system uses **tools** instead of embedding logic directly in agents.

Examples of custom tools:
- `compute_basic_stats()` – mean/min/max for each metric  
- `compute_correlations()` – checks relationships (e.g., sleep ↔ mood)  
- `recent_trend()` – extracts the last 7/14/30 days of data  
- `filter_by_date()` – optional date filtering  
- `generate_plot()` – code-based tool to create charts

Agents call these tools through a `tool_registry`, which imitates an MCP-style tool discovery mechanism.

---

### **4. Memory Layer**

#### **Session Memory**
- Tracks the current conversation.
- Stores partial results within a single agent run.

#### **Long-Term Memory**
- A simple JSON file (`wellness_memory.json`).
- Stores repeated or meaningful insights so the agent can re-use them across sessions.
- Demonstrates the concept of a lightweight **Memory Bank**.

---

### **5. Observability & Evaluation**

The system includes:

- **Logging & Tracing:**  
  Every agent action is recorded in a trace list with timestamps.

- **Metrics:**  
  For each run, the system records:
  - tool calls count,
  - runtime,
  - number of recommendations generated.

- **Evaluation Checks:**  
  Small tests verify:
  - the dataset loads,
  - the tools return expected outputs,
  - the agent produces suggestions.

These elements support debugging, transparency, and quality control, aligning with the course’s best practices for **agent observability**.

---

### **6. End-to-End Flow**

> User query  
> → PlannerAgent creates a plan  
> → AnalyticsAgent executes tools  
> → CoachAgent generates personalised recommendations  
> → Logs, memory updates, and metrics recorded  
> → Final wellness insights returned

This complete flow showcases how an AI agentic system can analyze wellness data and provide user-friendly, actionable insights.
